In [1]:
print("TEST...")

TEST...


In [2]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine


# Replace with your credentials
username = "postgres"
password = "ThePasswordis19!"
host = "localhost"
port = "5432"
database = "Vendor_Analysis"

# --- Create PostgreSQL connection engine ---
engine = create_engine(f'postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}')


In [7]:

# --- Load the aggregated table into a pandas DataFrame ---
df = pd.read_sql("SELECT * FROM vendor_sales_summary;", engine)


In [ ]:
# Quick look at our dataframe
df.head()
# first 5 and last 5 look the same from our PostgreSQL table too 
df.tail()

,VendorNumber,VendorName,Brand,Description,PurchasePrice,Actual_Price,Volume,Total_Purchase_Quantity,Total_Purchase_in_Dollars,Total_Sales_Quantity,Total_Sales_in_Dollars,Total_Sales_Price,Total_Excise_Tax,Freight_Cost
10687,10754,PERFECTA WINES,25381,Selbach-Oster Riesling,11.64,16.99,750,6.0,69.84,89.0,1512.11,1121.34,9.83,28720.52
10688,9744,FREDERICK WILDMAN & SONS,25425,La Scolca Black Label Gavi,24.82,35.99,750,12.0,297.84,12.0,431.88,35.99,1.35,3999.93
10689,9552,M S WALKER INC,25544,Tilia Bonarda Mendoza,7.94,11.99,750,1171.0,9297.74,807.0,8553.93,4977.35,89.61,55551.82
10690,10754,PERFECTA WINES,26442,Fortant Heritage Lang Rouge,10.80,15.99,750,2477.0,26751.60,1462.0,23377.38,11688.69,162.81,28720.52
10691,3252,E & J GALLO WINERY,17876,Darkhorse Cab Svgn,5.33,7.99,750,9368.0,49931.44,9179.0,73340.21,32511.31,1023.94,61966.91


In [ ]:
# should tell us all about data types stored in each column 
df.dtypes


VendorNumber                   int64
VendorName                    object
Brand                          int64
Description                   object
PurchasePrice                float64
Actual_Price                 float64
Volume                        object
Total_Purchase_Quantity      float64
Total_Purchase_in_Dollars    float64
Total_Sales_Quantity         float64
Total_Sales_in_Dollars       float64
Total_Sales_Price            float64
Total_Excise_Tax             float64
Freight_Cost                 float64
dtype: object

In [ ]:
# Lets check how many null values are present in our df


# AND Overall total amount of nulls within the whole df - so yea bunch of null values - present in our dataset
df.isnull().sum().sum()


# By columns - how many nulls
df.isnull().sum()

# NOTICE: 
# The columns that have a bunch of null values are these 4: 
# Total_Sales_Quantity, 
# Total_Sales_in_Dollars, 
# Total_Sales_Price,
# Total_Excise_Tax 


# WHAT MIGHT HAVE HAPPENED: 
# SO that means that the Vendor might've purchased them since Total_Purchase_Quantity - no nulls AND Total_Purchase_in_Dollars - no nulls either
# But most likely they were not sold


VendorNumber                   0
VendorName                     0
Brand                          0
Description                    0
PurchasePrice                  0
Actual_Price                   0
Volume                         0
Total_Purchase_Quantity        0
Total_Purchase_in_Dollars      0
Total_Sales_Quantity         178
Total_Sales_in_Dollars       178
Total_Sales_Price            178
Total_Excise_Tax             178
Freight_Cost                   0
dtype: int64

In [ ]:
df["VendorName"].unique() # Uniques are returned in order of appearance 

# SO there's cleary a bunch of irrelevant blank spaces added to all of the VendorNames for some reason 

array(['M S WALKER INC             ', 'PALM BAY INTERNATIONAL INC ',
       'THE IMPORTED GRAPE LLC     ', 'SOUTHERN WINE & SPIRITS NE ',
       'E & J GALLO WINERY         ', 'MARTIGNETTI COMPANIES',
       'PSP WINES ', 'SAZERAC CO INC             ',
       'CAMPARI AMERICA            ', 'PERFECTA WINES             ',
       'DIAGEO NORTH AMERICA INC   ', 'MAJESTIC FINE WINES        ',
       'JIM BEAM BRANDS COMPANY    ', 'STELLAR IMPORTING CO LLC   ',
       'MHW LTD                    ', 'WINE GROUP INC             ',
       'STATE WINE & SPIRITS   ', 'LATITUDE BEVERAGE COMPANY  ',
       'Circa Wines                ', 'CONSTELLATION BRANDS INC   ',
       'VINILANDIA USA             ', 'DELICATO VINEYARDS INC     ',
       'STE MICHELLE WINE ESTATES  ', 'TREASURY WINE ESTATES      ',
       'PROXIMO SPIRITS INC.       ', 'ULTRA BEVERAGE COMPANY LLP ',
       'BROWN-FORMAN CORP          ', 'PREMIER DISTRIBUTORS       ',
       'KOBRAND CORPORATION        ', 'PERNOD RICARD USA     

In [ ]:
df["Description"].unique() # No blank spaces here 

array(['Grahams 30-Yr Tawny Port', 'Bertani Amarone 04',
       'Matanzas Creek Chard Sonoma', ..., 'Tilia Bonarda Mendoza',
       'Fortant Heritage Lang Rouge', 'Darkhorse Cab Svgn'],
      shape=(9651,), dtype=object)

In [ ]:
df["Volume"].unique() # No blank spaces here: All unique values in Volume are numerical
# Volume is in an integer value - but for some reason the dtype is listed as an object - so we're gonna have to change it.

array(['750', '1500', '1000', '3000', '375', '1750', '50', '187', '4000',
       '5000', '330', '300', '200', '180', '500', '100', '20000', '150',
       '18000', '720', '400', '250', '162.5', '1100', '560', '6000',
       '600', '19500', '650', '3750', '9000'], dtype=object)

In [51]:
df[df[['Total_Sales_Price', 'Total_Sales_Quantity', 'Total_Sales_in_Dollars', 'Total_Excise_Tax']].isnull().any(axis=1)]

# so looking at the null rows - we can clearly see  Clarifies it for eg;
# VendorNumber : 4692 & VendorName: KOBRAND CORPORATION purchased 6.0 of Brand: 27069 Description: Medici Ermete Concerto Lmbrs 
# Paying Total_Purchase_in_Dollars: 80.52 
# but Total_Sales_Quantity, Total_Sales_in_Dollars, Total_Sales_Price, Total_Excise_Tax	for this product do not exist (are NaN)


,VendorNumber,VendorName,Brand,Description,PurchasePrice,Actual_Price,Volume,Total_Purchase_Quantity,Total_Purchase_in_Dollars,Total_Sales_Quantity,Total_Sales_in_Dollars,Total_Sales_Price,Total_Excise_Tax,Freight_Cost


In [ ]:
# SO
# 1. lets change the Volume dtype to float, 

# 2. remove random blank spaces in VendorNames,

# 3. AND change the null values to 0 since there are values present for Total_Purchase_Quantity AND Total_Purchase_in_Dollars 
#       BUT 
#    the # Total_Sales_Quantity, Total_Sales_in_Dollars, Total_Sales_Price, Total_Excise_Tax for them are missing 
#    so --> purchases were made but none were sold - so these values must/should be 0.


In [44]:
df["Volume"] = df["Volume"].astype('float64')



In [46]:
df.dtypes # so we can see that it has been changed: Volume                       float64


VendorNumber                   int64
VendorName                    object
Brand                          int64
Description                   object
PurchasePrice                float64
Actual_Price                 float64
Volume                       float64
Total_Purchase_Quantity      float64
Total_Purchase_in_Dollars    float64
Total_Sales_Quantity         float64
Total_Sales_in_Dollars       float64
Total_Sales_Price            float64
Total_Excise_Tax             float64
Freight_Cost                 float64
dtype: object

In [47]:
df.fillna(0, inplace=True)

In [ ]:
# lets check again 

df[df[["Total_Sales_Quantity", "Total_Sales_in_Dollars", "Total_Sales_Price", "Total_Excise_Tax"]].isnull().any(axis=1)]

# so we can see the output returns only column headers - no values - NONE at all are null anymore

# OR we can simply do this

df.isnull().sum().sum() # this has given us --> np.int64(0)

df.isnull().sum() # AND this gives us 0 in each column

VendorNumber                 0
VendorName                   0
Brand                        0
Description                  0
PurchasePrice                0
Actual_Price                 0
Volume                       0
Total_Purchase_Quantity      0
Total_Purchase_in_Dollars    0
Total_Sales_Quantity         0
Total_Sales_in_Dollars       0
Total_Sales_Price            0
Total_Excise_Tax             0
Freight_Cost                 0
dtype: int64

In [53]:
df["VendorName"] = df["VendorName"].str.strip()

In [ ]:
df["VendorName"].unique() # so we can see blank spaces have been stripped away.

array(['M S WALKER INC', 'PALM BAY INTERNATIONAL INC',
       'THE IMPORTED GRAPE LLC', 'SOUTHERN WINE & SPIRITS NE',
       'E & J GALLO WINERY', 'MARTIGNETTI COMPANIES', 'PSP WINES',
       'SAZERAC CO INC', 'CAMPARI AMERICA', 'PERFECTA WINES',
       'DIAGEO NORTH AMERICA INC', 'MAJESTIC FINE WINES',
       'JIM BEAM BRANDS COMPANY', 'STELLAR IMPORTING CO LLC', 'MHW LTD',
       'WINE GROUP INC', 'STATE WINE & SPIRITS',
       'LATITUDE BEVERAGE COMPANY', 'Circa Wines',
       'CONSTELLATION BRANDS INC', 'VINILANDIA USA',
       'DELICATO VINEYARDS INC', 'STE MICHELLE WINE ESTATES',
       'TREASURY WINE ESTATES', 'PROXIMO SPIRITS INC.',
       'ULTRA BEVERAGE COMPANY LLP', 'BROWN-FORMAN CORP',
       'PREMIER DISTRIBUTORS', 'KOBRAND CORPORATION', 'PERNOD RICARD USA',
       'DJINN SPIRITS LLC', 'MOET HENNESSY USA INC', 'LUXCO INC',
       'SAZERAC NORTH AMERICA INC.', 'CANDIA VINEYARDS',
       'WILLIAM GRANT & SONS INC', 'SURVILLE ENTERPRISES CORP',
       'BACARDI USA INC', 'PINE

In [59]:
# NOW we'll create some new columns in our df (or simply the vendor_performance_summary table)  
# which would be beneficial for our Analysis (EDA & Research questions + Testing) and even for our Power BI DashBoard later on

In [ ]:
# We'll create these 4 new columns: Gross_Porfit, Profit_Margin, Stock_Turnover, Sales_to_Purchase_Ratio

df["Gross_Profit"] = df["Total_Sales_in_Dollars"] - df["Total_Purchase_in_Dollars"]

In [70]:
df["Profit_Margin"] = (df["Gross_Profit"]/df["Total_Sales_in_Dollars"].replace(0, np.nan))*100

In [73]:
df["Sales_to_Purchase_Ratio"] = (df["Total_Sales_in_Dollars"]/df["Total_Purchase_in_Dollars"].replace(0, np.nan))

In [ ]:
# So we just added all of these: 
'''
df["Gross_Profit"] = df["Total_Sales_in_Dollars"] - df["Total_Purchase_in_Dollars"]
df["Profit_Margin"] = (df["Gross_Profit"]/df["Total_Sales_in_Dollars"].replace(0, np.nan))*100
df["Stock_Turnover"] = (df["Total_Sales_Quantity"]/df["Total_Purchase_Quantity"].replace(0, np.nan))
df["Sales_to_Purchase_Ratio"] = (df["Total_Sales_in_Dollars"]/df["Total_Purchase_in_Dollars"].replace(0, np.nan))  
'''

In [74]:
df # so yup created these new columns - DONE


,VendorNumber,VendorName,Brand,Description,PurchasePrice,Actual_Price,Volume,Total_Purchase_Quantity,Total_Purchase_in_Dollars,Total_Sales_Quantity,Total_Sales_in_Dollars,Total_Sales_Price,Total_Excise_Tax,Freight_Cost,Gross_Profit,Profit_Margin,Stock_Turnover,Sales_to_Purchase_Ratio
0,9552,M S WALKER INC,10937,Grahams 30-Yr Tawny Port,61.64,89.99,750.0,15.0,924.60,15.0,1349.85,899.90,1.66,55551.82,425.25,31.503500,1.000000,1.459929
1,6785,PALM BAY INTERNATIONAL INC,18433,Bertani Amarone 04,62.91,94.99,750.0,7.0,440.37,63.0,5984.37,4654.51,6.93,13605.41,5544.00,92.641331,9.000000,13.589413
2,9552,M S WALKER INC,36571,Matanzas Creek Chard Sonoma,16.33,25.99,750.0,12.0,195.96,21.0,543.79,334.87,2.34,55551.82,347.83,63.964030,1.750000,2.775005
3,90017,THE IMPORTED GRAPE LLC,45577,Pacifico Sur Chard Curico,2.80,5.29,750.0,72.0,201.60,69.0,503.01,87.48,7.75,80.03,301.41,59.921274,0.958333,2.495089
4,2000,SOUTHERN WINE & SPIRITS NE,32137,F Coppola Diamond Znfd,9.39,15.99,750.0,1334.0,12526.26,1438.0,21723.62,15630.68,158.93,19016.59,9197.36,42.338063,1.077961,1.734246
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10687,10754,PERFECTA WINES,25381,Selbach-Oster Riesling,11.64,16.99,750.0,6.0,69.84,89.0,1512.11,1121.34,9.83,28720.52,1442.27,95.381288,14.833333,21.651060
10688,9744,FREDERICK WILDMAN & SONS,25425,La Scolca Black Label Gavi,24.82,35.99,750.0,12.0,297.84,12.0,431.88,35.99,1.35,3999.93,134.04,31.036399,1.000000,1.450040
10689,9552,M S WALKER INC,25544,Tilia Bonarda Mendoza,7.94,11.99,750.0,1171.0,9297.74,807.0,8553.93,4977.35,89.61,55551.82,-743.81,-8.695535,0.689155,0.920001
10690,10754,PERFECTA WINES,26442,Fortant Heritage Lang Rouge,10.80,15.99,750.0,2477.0,26751.60,1462.0,23377.38,11688.69,162.81,28720.52,-3374.22,-14.433696,0.590230,0.873868


In [80]:
df.isnull().sum()

VendorNumber                   0
VendorName                     0
Brand                          0
Description                    0
PurchasePrice                  0
Actual_Price                   0
Volume                         0
Total_Purchase_Quantity        0
Total_Purchase_in_Dollars      0
Total_Sales_Quantity           0
Total_Sales_in_Dollars         0
Total_Sales_Price              0
Total_Excise_Tax               0
Freight_Cost                   0
Gross_Profit                   0
Profit_Margin                178
Stock_Turnover                 0
Sales_to_Purchase_Ratio        0
dtype: int64

In [ ]:
df["Profit_Margin"].fillna(0, inplace=True)


In [81]:
df["Profit_Margin"].fillna(0, inplace=True)

C:\Users\shona\AppData\Local\Temp\ipykernel_1948\1511866893.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["Profit_Margin"].fillna(0, inplace=True)


In [82]:
df.isnull().sum()

VendorNumber                 0
VendorName                   0
Brand                        0
Description                  0
PurchasePrice                0
Actual_Price                 0
Volume                       0
Total_Purchase_Quantity      0
Total_Purchase_in_Dollars    0
Total_Sales_Quantity         0
Total_Sales_in_Dollars       0
Total_Sales_Price            0
Total_Excise_Tax             0
Freight_Cost                 0
Gross_Profit                 0
Profit_Margin                0
Stock_Turnover               0
Sales_to_Purchase_Ratio      0
dtype: int64

In [83]:
df["Profit_Margin"] = df["Profit_Margin"].fillna(0)

In [84]:
df.isnull().sum()

VendorNumber                 0
VendorName                   0
Brand                        0
Description                  0
PurchasePrice                0
Actual_Price                 0
Volume                       0
Total_Purchase_Quantity      0
Total_Purchase_in_Dollars    0
Total_Sales_Quantity         0
Total_Sales_in_Dollars       0
Total_Sales_Price            0
Total_Excise_Tax             0
Freight_Cost                 0
Gross_Profit                 0
Profit_Margin                0
Stock_Turnover               0
Sales_to_Purchase_Ratio      0
dtype: int64

In [87]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
VendorNumber,10692.0,10650.649458,18753.519148,2.000000,3951.000000,7153.000000,9552.000000,2.013590e+05
Brand,10692.0,18039.228769,12662.187074,58.000000,5793.500000,18761.500000,25514.250000,9.063100e+04
PurchasePrice,10692.0,24.385303,109.269375,0.360000,6.840000,10.455000,19.482500,5.681810e+03
Actual_Price,10692.0,35.643671,148.246016,0.490000,10.990000,15.990000,28.990000,7.499990e+03
Volume,10692.0,847.360550,664.309212,50.000000,750.000000,750.000000,750.000000,2.000000e+04
Total_Purchase_Quantity,10692.0,3140.886831,11095.086769,1.000000,36.000000,262.000000,1975.750000,3.376600e+05
Total_Purchase_in_Dollars,10692.0,30106.693372,123067.799627,0.710000,453.457500,3655.465000,20738.245000,3.811252e+06
Total_Sales_Quantity,10692.0,3077.482136,10952.851391,0.000000,33.000000,261.000000,1929.250000,3.349390e+05
Total_Sales_in_Dollars,10692.0,42239.074419,167655.265984,0.000000,729.220000,5298.045000,28396.915000,5.101920e+06
Total_Sales_Price,10692.0,18793.783627,44952.773386,0.000000,289.710000,2857.800000,16059.562500,6.728193e+05


In [91]:
df[df["Total_Sales_Quantity"] < 0]


,VendorNumber,VendorName,Brand,Description,PurchasePrice,Actual_Price,Volume,Total_Purchase_Quantity,Total_Purchase_in_Dollars,Total_Sales_Quantity,Total_Sales_in_Dollars,Total_Sales_Price,Total_Excise_Tax,Freight_Cost,Gross_Profit,Profit_Margin,Stock_Turnover,Sales_to_Purchase_Ratio


In [92]:
df[df["Total_Purchase_Quantity"] < 0] 

,VendorNumber,VendorName,Brand,Description,PurchasePrice,Actual_Price,Volume,Total_Purchase_Quantity,Total_Purchase_in_Dollars,Total_Sales_Quantity,Total_Sales_in_Dollars,Total_Sales_Price,Total_Excise_Tax,Freight_Cost,Gross_Profit,Profit_Margin,Stock_Turnover,Sales_to_Purchase_Ratio


In [ ]:
df.duplicated().sum() # SO --> We have now confirmed that NO duplicate reocrds are present either 
# --> SO lets bring this cleaned table (With 4 new columns) back to our PostgreSQL Db  

np.int64(0)

In [95]:
# So our engine has already been created variable engine - create engine - DONE -- now all we gotta do is push our df into the PostgreSQL Db

df.to_sql("vendor_sales_summary_clean_addedcols", engine, if_exists="replace", index=False)

print("Done, Added succesfully!")

Done, Added succesfully!
